# Make reference .geno  
Make with all SNPs, and with just V4 filtering

In [1]:
import pandas as pd
import numpy as np

In [2]:
infile1 = "refs_MA_52_0120.xlsx"
infile2 = "refs_MA_39_0120.xlsx"

In [3]:
# read in Genotypes sheet from excel file 
inxl52 = pd.ExcelFile(infile1)
inxl39 = pd.ExcelFile(infile2)

In [4]:
db52_geno = inxl52.parse('Genotypes')
db52_geno['Date'] = db52_geno['Date'].apply(str)

In [5]:
db39_geno = inxl39.parse('Genotypes')
db39_geno['Date'] = db39_geno['Date'].apply(str)

In [6]:
db52_39_geno = db52_geno.merge(db39_geno,on=["Date","SAMPLE_NAME","WELL"])
db52_39_geno.set_index('SAMPLE_NAME', drop = False, inplace=True)
db52_39_geno.shape

(73, 94)

## V4

In [7]:
import re
f = open("../Running_Panel/V4badSNPs","r")
badsnps=f.readline().strip()
badsnps=re.split('\s-s\s|-s\s',badsnps)[1:]
f.close()
#print(re.split('XXX|YYY|ZZZ', s_strs))
badsnps

['X126230_13777',
 'X141791_7873',
 'X190236_2746',
 'X194561_9140',
 'X194968_5912',
 'X2156_1716',
 'X218289_7454',
 'X221258_4404',
 'X117099_13373',
 'X201239_10000',
 'X205485_3157',
 'X230449_27274',
 'X6349_7011',
 'X112084_4248',
 'X228046_17494',
 'X224779_3298',
 'X195403_4772',
 'X247020_27217']

In [8]:
db52_39_sfilt = db52_39_geno.drop(labels=badsnps,axis=1)
db52_39_sfilt.shape

(73, 76)

In [9]:
df = db52_39_sfilt
coverage_row= df.iloc[:,3:].count(axis=1)/73
df_coverage=pd.DataFrame(data=coverage_row,columns=['Coverage'])
df52_39_sfilt =pd.concat([df,df_coverage],axis=1)
df52_39_sfilt.head()

Date  SAMPLE_NAME WELL X104650_2998 X104795_7287 X107554_774  \
SAMPLE_NAME                                                                  
CAN11619rB   52920   CAN11619rB  E12            T            G           T   
CANOT11829   21920   CANOT11829  C06            T            G           T   
CANOT11829B  21920  CANOT11829B  D04            T            G           T   
CANOT11830   21920   CANOT11830  H03            T            G           T   
CANOT11830B  32720  CANOT11830B  H07            T            G           T   

            X120320_1376 X121338_5301 X135849_12666 X136296_2709  ...  \
SAMPLE_NAME                                                       ...   
CAN11619rB             A            G             T            G  ...   
CANOT11829             A            G             T            G  ...   
CANOT11829B            A            G             T            G  ...   
CANOT11830             A            G             T            G  ...   
CANOT11830B            A            G             T            G  ...   

            X235476_9196 X236674_44316 X241030_2917 X242562_17320  \
SAMPLE_NAME                                                         
CAN11619rB             G             T            C             G   
CANOT11829             G             T            C             G   
CANOT11829B            G             T            C             G   
CANOT11830             G             T            C             G   
CANOT11830B            G             T            C             G   

            X243446_17343 X246396_2361 X247085_6096 X99556_26307 X99669_18498  \
SAMPLE_NAME                                                                     
CAN11619rB              C            A            A            T           TA   
CANOT11829              C            A            A            T            A   
CANOT11829B             C            A            A            T            A   
CANOT11830              C            A            A            T            A   
CANOT11830B             C            A            A            T            A   

             Coverage  
SAMPLE_NAME            
CAN11619rB   1.000000  
CANOT11829   1.000000  
CANOT11829B  0.986301  
CANOT11830   1.000000  
CANOT11830B  1.000000  

[5 rows x 77 columns]

In [10]:
failedDF = df52_39_sfilt[df52_39_sfilt['Coverage'] < 0.93]
failed_sample = list(failedDF["SAMPLE_NAME"])
failed_sample

['CANOT11837', 'SL04', 'SL32']

In [11]:
passDF = df52_39_sfilt[df52_39_sfilt['Coverage'] >= 0.93]

In [12]:
passDF.iloc[50:,[0,76]].sort_values(by='SAMPLE_NAME')

Date  Coverage
SAMPLE_NAME                 
MATT01           X  0.986301
MATT04           X  1.000000
MATT08b          X  1.000000
MATT09           X  1.000000
Mill12           X  1.000000
Mill6            X  1.000000
Mill8            X  1.000000
Otter13          X  0.986301
Otter16          X  1.000000
Otter17          X  1.000000
Otter21          X  1.000000
Otter23          X  0.958904
Otter40          X  1.000000
SIPS114          X  1.000000
Sipsey117        X  0.986301
Sipsey120        X  0.986301
Sipsey125        X  1.000000
Sipsey126        X  1.000000
TN11853      32720  0.986301
TN11854      32720  0.986301

In [13]:
badsamps = ["CANOT11829B","CANOT11830","FL16",
            "FLM13B","SL10B","SL14",'SL32', 'SL20','SL31','CANOT11837', 'SL04']

### Make V4 geno for now

In [14]:
db52_39_filt = df52_39_sfilt.drop(badsamps) 

In [15]:
db52_39_filt.drop(labels=['Date','WELL','SAMPLE_NAME','Coverage'],
          axis=1,inplace=True)
db52_39_filt.shape

(62, 73)

In [16]:
alleles = {'A':'0101','T':'0404','C':'0202','G':'0303',
         'TA':'0401','AT':'0104',
        'CT':'0204','TC':'0402',
        'GC':'0302','CG':'0203',
        'TG':'0403','GT':'0304',
        'CA':'0201','AC':'0102',
          'GA':'0301','AG':'0103'}
db52_39_filt = db52_39_filt.replace(to_replace=alleles)
db52_39_filt = db52_39_filt.replace(np.nan, '0000', regex=True)

In [17]:
db52_39_filt.rename(index=lambda x: x+"rf,", inplace=True)


In [18]:
final = open("refV4_0120.gen","w")
final.write("MA\n")
final.write(',\t'.join(list(db52_39_filt.columns))+"\n")
final.write("Pop\n")
final.write(db52_39_filt.to_csv(header=False, sep = "\t"))
final.close()

## V5

In [8]:
import re
f = open("../Running_Panel/V5badSNPs","r")
badsnps = []
for line in f:
    badsnps.append(line.strip())
f.close()
#print(re.split('XXX|YYY|ZZZ', s_strs))
badsnps

['X126230_13777',
 'X141791_7873',
 'X190236_2746',
 'X194561_9140',
 'X194968_5912',
 'X2156_1716',
 'X218289_7454',
 'X221258_4404',
 'X117099_13373',
 'X201239_10000',
 'X205485_3157',
 'X230449_27274',
 'X6349_7011',
 'X112084_4248',
 'X228046_17494',
 'X224779_3298',
 'X195403_4772',
 'X247020_27217',
 'X126488_13871',
 'X188611_1228',
 'X232694_38736',
 'X219104_2929']

In [9]:
db52_39_sfilt = db52_39_geno.drop(labels=badsnps,axis=1)
db52_39_sfilt.shape

(59, 72)

In [11]:
df = db52_39_sfilt
coverage_row= df.iloc[:,3:].count(axis=1)/69
df_coverage=pd.DataFrame(data=coverage_row,columns=['Coverage'])
df52_39_sfilt =pd.concat([df,df_coverage],axis=1)
df52_39_sfilt.head()

Date SAMPLE_NAME WELL X104650_2998 X104795_7287 X107554_774  \
SAMPLE_NAME                                                                 
CAN11619rB   52920  CAN11619rB  E12            T            G           T   
DB60778B     52920    DB60778B  F03            T            G           T   
SL14r        52920       SL14r  A04            C            A           T   
SL32         52920        SL32  F04          NaN           GA           T   
SL34         52920        SL34  H09            C           GA           T   

            X120320_1376 X121338_5301 X135849_12666 X136296_2709  ...  \
SAMPLE_NAME                                                       ...   
CAN11619rB             A            G             T            G  ...   
DB60778B               A            G             T            G  ...   
SL14r                  G            A             T            G  ...   
SL32                   G           AG           NaN          NaN  ...   
SL34                   G            A             T            G  ...   

            X235476_9196 X236674_44316 X241030_2917 X242562_17320  \
SAMPLE_NAME                                                         
CAN11619rB             G             T            C             G   
DB60778B               G             T            C             G   
SL14r                  A             T            C             A   
SL32                 NaN             T          NaN            AG   
SL34                   A             T            C             A   

            X243446_17343 X246396_2361 X247085_6096 X99556_26307 X99669_18498  \
SAMPLE_NAME                                                                     
CAN11619rB              C            A            A            T           TA   
DB60778B                C            A            A            T            A   
SL14r                   C            A            G            C            T   
SL32                    C            A          NaN            C            T   
SL34                    C            A            G            C            T   

             Coverage  
SAMPLE_NAME            
CAN11619rB   1.000000  
DB60778B     1.000000  
SL14r        0.985507  
SL32         0.753623  
SL34         1.000000  

[5 rows x 73 columns]

In [12]:
failedDF = df52_39_sfilt[df52_39_sfilt['Coverage'] < 0.93]
failed_sample = list(failedDF["SAMPLE_NAME"])
failed_sample

['SL32', 'CANOT11837', 'SL04']

In [13]:
passDF = df52_39_sfilt[df52_39_sfilt['Coverage'] >= 0.93]

In [17]:
passDF.iloc[:,[0,72]].sort_values(by='SAMPLE_NAME')

Date  Coverage
SAMPLE_NAME                 
CAN11619rB   52920  1.000000
CANOT11829   21920  1.000000
CANOT11829B  21920  0.985507
CANOT11830   21920  1.000000
CANOT11830B  32720  1.000000
CANOT11832   21920  0.985507
CANOT11838   21920  1.000000
CANOT2617    21920  0.971014
CANOT2618    21920  0.985507
DB12041      21920  1.000000
DB2589       21920  1.000000
DB2590       21920  0.985507
DB2592       21920  1.000000
DB2593       21920  0.971014
DB2634       21920  0.971014
DB2634B      21920  0.971014
DB2635       21920  1.000000
DB2637       21920  1.000000
DB60778B     52920  1.000000
FL01         21920  1.000000
FL09         21920  1.000000
FL12         21920  1.000000
FL15         21920  0.985507
FL16         21920  0.985507
FL16B        32720  1.000000
FL20         21920  1.000000
FL21         21920  1.000000
FL23         21920  1.000000
FL30         21920  0.985507
FLF17        21920  1.000000
FLM10        21920  1.000000
FLM12        21920  1.000000
FLM13        21920  1.000000
FLM13B       21920  0.985507
FLM15        21920  1.000000
SL10         21920  0.956522
SL10B        32720  0.971014
SL11         21920  0.942029
SL14         21920  0.971014
SL14r        52920  0.985507
SL15         21920  0.971014
SL16         21920  0.985507
SL19         21920  0.971014
SL19B        21920  0.956522
SL2          21920  0.971014
SL20         52920  0.956522
SL31         52920  0.942029
SL34         52920  1.000000
SL36         52920  1.000000
SL37         52920  1.000000
SL40         52920  0.985507
SL43         52920  1.000000
SL8          21920  0.971014
TN11851      32720  1.000000
TN11853      32720  0.985507
TN11854      32720  0.985507

In [18]:
badsamps = ["CANOT11829B","CANOT11830","DB2634B","FL16",
            "FLM13B","SL10","SL14","SL19B",'SL32', 'CANOT11837', 'SL04']

In [19]:
db52_39_filt = df52_39_sfilt.drop(badsamps) 

In [20]:
db52_39_filt.drop(labels=['Date','WELL','SAMPLE_NAME','Coverage'],
          axis=1,inplace=True)
db52_39_filt.shape

(48, 69)

In [21]:
alleles = {'A':'0101','T':'0404','C':'0202','G':'0303',
         'TA':'0401','AT':'0104',
        'CT':'0204','TC':'0402',
        'GC':'0302','CG':'0203',
        'TG':'0403','GT':'0304',
        'CA':'0201','AC':'0102',
          'GA':'0301','AG':'0103'}
db52_39_filt = db52_39_filt.replace(to_replace=alleles)
db52_39_filt = db52_39_filt.replace(np.nan, '0000', regex=True)

In [22]:
db52_39_filt.rename(index=lambda x: x+",", inplace=True)


In [23]:
final = open("refV5.gen","w")
final.write("MA\n")
final.write(',\t'.join(list(db52_39_filt.columns))+"\n")
final.write("Pop\n")
final.write(db52_39_filt.to_csv(header=False, sep = "\t"))
final.close()